#### The Model Training Module development

In [1]:
import os

print(f"Original Working Directory:{os.getcwd()}")

os.chdir("../")

print(f"Current Working Directory after Changed: {os.getcwd()}")

Original Working Directory:c:\Users\Olanrewaju Adegoke\DLOPs\Kidney_Disease_Classification\research
Current Working Directory after Changed: c:\Users\Olanrewaju Adegoke\DLOPs\Kidney_Disease_Classification


In [2]:
# UPDATE ENTITY - AFTER THE CONFIG AND PARAMS ARE UPDATED

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingModelConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_augmentation: bool
    params_image_size: list
    params_batch_size: int
    params_epochs: int



In [3]:
# UPDATE THE CONFIGURATION INSIDE THE SRC/CONFIG - THIS IS ESSENTIAL AS THE CURRENT WORKFLOW WILL USE IT TO RUN

from src.Kidney_classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

#from src.Kidney_classifier.constants import *

from Kidney_classifier.utils.common import read_yaml_file, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):


        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)

        create_directories([self.config.artifacts_root])

    
    # def get_training_model_config(self) -> TrainingModelConfig:
    #     config_for_training = self.config.training_model
    #     params_for_training = self.params.training_model
    #     prepare_base_model = self.config.prepare_base_model
    #     training_data_access = os.path.join(self.config.data_ingestion.unzip_dir)

    #     create_directories([Path(config_for_training.root_dir)])

    #     training_model_config = TrainingModelConfig(
    #         root_dir=Path(config_for_training.root_dir),
    #         trained_model_path=Path(config_for_training.trained_model_path),
    #         updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
    #         training_data=Path(training_data_access),
    #         params_augmentation=params_for_training.AUGMENTATION,
    #         params_image_size=params_for_training.IMAGE_SIZE,
    #         params_batch_size=params_for_training.BATCH_SIZE,
    #         params_epochs=params_for_training.EPOCHS
    #     )

    def get_training_model_config(self) -> TrainingModelConfig:
        config_for_training = self.config.training_model
        prepare_base_model = self.config.prepare_base_model
        params = self.params.prepare_base_model
        training_data_access = os.path.join(self.config.data_ingestion.unzip_dir)

        create_directories([Path(config_for_training.root_dir)])

        training_model_config = TrainingModelConfig(
            root_dir=Path(config_for_training.root_dir),
            trained_model_path=Path(config_for_training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data_access),
            params_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_batch_size=params.BATCH_SIZE,
            params_epochs=params.EPOCHS
        )

        return training_model_config

In [4]:
# UPDATE THE COMPONENTS - THIS IS WHERE YOU CREATE THE FILE FOR TRAINING THE MODEL

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
tf.config.run_functions_eagerly(True)
import time
from Kidney_classifier import logger

In [5]:
class ModelTraining:
    def __init__(self, config: TrainingModelConfig):
        self.config = config

    
    def get_updated_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

        # YOU NEED TO ALWAYS RECOMPILE YOUR MODEL FOR TRANSFER LEARNING (wHEN YOU LOAD A KERAS MODEL ALWAYS RE-COMPILE)
        optimizer = Adam()
        self.model.compile(
            optimizer=optimizer,
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

    
    def training_validation_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.10
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        validation_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.validation_generator = validation_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_augmentation:
            training_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            training_datagenerator = validation_datagenerator

        self.training_generator = training_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def model_training(self):
        self.steps_per_epoch = self.training_generator.samples // self.training_generator.batch_size
        self.validation_steps = self.validation_generator.samples // self.validation_generator.batch_size

        self.model.fit(
            self.training_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.validation_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [6]:
# UPDATE THE PIPELINE

try:
    config = ConfigurationManager()
    training_model_config = config.get_training_model_config()
    training_model = ModelTraining(config=training_model_config)
    training_model.get_updated_base_model()
    training_model.training_validation_generator()
    training_model.model_training()

except Exception as e:
    raise e

[2025-08-12 19:45:17,699: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-12 19:45:17,704: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-12 19:45:17,707: INFO: common: created directory at: artifacts]
[2025-08-12 19:45:17,715: INFO: common: created directory at: artifacts\training]
[2025-08-12 19:45:18,427: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 40 images belonging to 2 classes.
Found 360 images belonging to 2 classes.


c:\Users\Olanrewaju Adegoke\DLOPs\Kidney_Disease_Classification\KIDNEY_CLASS_VENV\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\Olanrewaju Adegoke\DLOPs\Kidney_Disease_Classification\KIDNEY_CLASS_VENV\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


22/22 ━━━━━━━━━━━━━━━━━━━━ 387s 18s/step - accuracy: 0.7006 - loss: 0.8907 - val_accuracy: 1.0000 - val_loss: 0.0806
[2025-08-12 19:51:46,538: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
